In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome,ChromeOptions
import json
import time
import warnings
import ddddocr
import base64
import re

ocr = ddddocr.DdddOcr()

warnings.filterwarnings("ignore")

def password():
    pass
def node_exist(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
        return True
    except:
        return False
def downloader(subdriver, href):
    follow = subdriver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/div/div[2]/div[1]/div/div[1]/div[2]/div/button')
    if follow.text != '已关注':
        follow.click()
    time.sleep(0.5)
    like = subdriver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/div/div[1]/div[3]/div/div/div[1]/div/div[1]/div[1]/div[1]')
    if like.get_attribute('class') != 'hover_mouse p_font_md':
        like.click()
    time.sleep(0.5)
    collect = subdriver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/div/div[1]/div[3]/div/div/div[1]/div/div[1]/div[1]/div[2]')
    if collect.get_attribute('class') != 'margin_left_big hover_mouse p_font_md':
        collect.click()
    time.sleep(0.5)
    download = subdriver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/div/div[2]/div[3]/div[2]/button')
    download.click()
    time.sleep(2) # 必要sleep，加载验证信息
    
    # 密码尚未适配
    if(node_exist(subdriver, '/html/body/div[1]/section/section/main/div/div[5]/div/div[2]/div/div/div[2]/input')):
        print('FAILED: ', href)
        return
    
    # 验证码OCR识别
    if(node_exist(subdriver, '/html/body/div[1]/section/section/main/div/div[8]/div/div[2]/div/div/img')):
        print('VALID CODE')
        code = subdriver.find_element(By.XPATH, '/html/body/div[1]/section/section/main/div/div[8]/div/div[2]/div/div/img')
        base64_code = code.get_attribute('src')
        result = re.search("data:image/(?P<ext>.*?);base64,(?P<data>.*)", base64_code, re.DOTALL)
        data = result.groupdict().get("data")
        ext = result.groupdict().get("ext")
#         print(data)
#         img_data = base64.b64decode(base64_code)
#         img_array = np.fromstring(img_data, np.uint8)
#         img = cv2.imdecode(img_array, cv2.COLOR_RGB2BGR)
        img = base64.urlsafe_b64decode(data)
        res = ocr.classification(img)
        print(res)
        time.sleep(2)
        text = subdriver.find_element(By.XPATH, '/html/body/div[1]/section/section/main/div/div[8]/div/div[2]/div/div/div/input')
        text.send_keys(res)
        time.sleep(1)
        submit = subdriver.find_element(By.XPATH, '/html/body/div[1]/section/section/main/div/div[8]/div/div[3]/div/button')
        submit.click()
        time.sleep(0.5) # 必要sleep
        if node_exist(subdriver, '/html/body/div[2]'): #验证码错误alert
            print('FAILED: ', href)
    time.sleep(3)

In [ ]:
# response
driver_path = './chromedriver.exe' 
broswer_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"  
Options = ChromeOptions()
Options.binary_location = broswer_path
Options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"')
driver = Chrome(chrome_options=Options, executable_path = driver_path)
# cookies_path=r'cookies.txt'
# with open(cookies_path,'r',encoding='utf-8') as f:
#     cookies_str=f.read()
# cookies_list=json.loads(cookies_str)

url='https://www.aplaybox.com/login'
driver.get(url)
# time.sleep(3)

In [ ]:
# login
# driver.refresh()
time.sleep(2)
usr = driver.find_element(By.XPATH, '/html/body/div/div/div[1]/div[1]/div/div[2]/form/div[1]/div/div/input')
usr.send_keys('') # 账号
pwd = driver.find_element(By.XPATH, '/html/body/div/div/div[1]/div[1]/div/div[2]/form/div[2]/div/div[1]/input')
pwd.send_keys('') # 密码
chk = driver.find_element(By.XPATH, '/html/body/div/div/div[1]/div[1]/div/div[2]/form/label/span[1]')
chk.click() # 守则
time.sleep(0.5)
login = driver.find_element(By.XPATH, '/html/body/div/div/div[1]/div[1]/div/div[2]/form/div[4]/div/div/button[1]')
login.click() # 登录
time.sleep(3)

In [ ]:
url='https://www.aplaybox.com/model/model'
driver.get(url)
time.sleep(2)
filter1 = driver.find_element(By.XPATH, '/html/body/div[1]/section/section/main/div/div[1]/div[2]/div[1]/div/div[1]/div[3]/div[3]/div')
filter1.click() # 人物
time.sleep(1)
filter1 = driver.find_element(By.XPATH, '/html/body/div[1]/section/section/main/div/div[1]/div[2]/div[1]/div/div[1]/div[5]/div[4]')
filter1.click() # 最多下载
time.sleep(1)
chk = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/div[2]/div[1]/div/div[1]/div[5]/div[6]/label/span[1]')
chk.click() # 允许下载
time.sleep(2)

In [ ]:
prefix = '/html/body/div[1]/section/section/main/div/div[1]/div[2]/div[1]/div/div[2]'
for p in range(3): # 爬多少页
    for i in range(9): # 默认一页9张，根据默认数量调整
        xpath = prefix + f'/div[{i+1}]/a[1]'
        href = driver.find_element(By.XPATH, xpath).get_attribute('href')
#         href = 'https://www.aplaybox.com/details/model/GeEjGX7sQ9Q6'
    #     print(href)

        js = f'window.open("{href}");'
        driver.execute_script(js)

    #     print(driver.current_window_handle) 
        handles = driver.window_handles
    #     print(handles)

        for handle in handles:
            if handle != driver.current_window_handle:
    #             print('switch to ',handle)
                driver.switch_to.window(handle)
    #             print(driver.current_window_handle)
                break
        time.sleep(5) # 必要sleep，加载下载页
        downloader(driver, href)
        time.sleep(5)
        driver.close()
        driver.switch_to.window(handles[0])
    #     print(handles)
    #     break
    next_page = driver.find_element(By.XPATH, '/html/body/div/section/section/main/div/div[1]/div[2]/div[1]/div/div[3]/div/button[2]')
    next_page.click()
    time.sleep(2)